# Scraping Data 


## In this notebook, I am going to scrape data from Somon.tj and save it into a CSV file.

## For this project we need the following libraries:
<ul>

<li>requests</li>
<p>We use requests library for sending requests to the server and getting the response as raw HTML file.</p>
<li>Beautifull Soup</li>
<p>Beatiful Soup is used for scraping the HTML file, to extract some specific data from it.</p>
<li>csv</li>
<p>From csv we use writer function to add new rows to our data (CSV file).</p>
</ul>

In [1]:
##Importing the libraries

from csv import writer
import requests
from bs4 import BeautifulSoup
from bs4 import element
print("The libraries are successfully imported!")

The libraries are successfully imported!


<p> Once we have the libraries imported, then we specify the skeliton of our URL which contains the service type "nedvizhimost" (real estate), subservice "prodazha-kvartir" (apartment selling), and the city (Dushanbe). As the number of apartments on sale in dushanbe do not fill in one page there will be many pages. And going to each of those pages, we specify them in"?page=(# of the page)". </br>
<img src="pages.PNG" stype="display: block; margin-left: auto; margin-right: auto;  width: 50%;"></br>
Next we pass the first page to request.get(), to get its content as an html file and buy parsing the ul of class "number-list". Then convert it into a list of integers and get the last element which shows the number of the pages.
</p>



In [2]:
URL = "https://somon.tj/nedvizhimost/prodazha-kvartir/dushanbe/?page="
page = requests.get(URL+str(1))
soup = BeautifulSoup(page.content,'html.parser')
result = soup.find("ul",class_="number-list")
pages = [int(item.find("a").text) for item in result if item.find("a") is not None and isinstance(item,element.Tag)][-1]

print(f"There are {pages} pages for parsing.")

There are 123 pages for parsing.


<p>From the above code we find how many pages we have. The below function is applied on each page to get basic information like number_of_rooms,floor,area,address,date and price of each apartment and add it as a row to our csv file.</p>

In [5]:
def page_parser(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    result = soup.find("ul",class_="list-simple__output")
    for item in result:
        if isinstance(item,element.Tag):
            meta = item.find("a").text.strip().split(",")
            print(meta)
            if "-" in meta[0]:
                number_of_rooms = int(meta[0].split("-")[0])
            else:
                number_of_rooms = int(meta[0].split(" ")[0])
            if meta[1].split()[0].isnumeric():
                if meta[1].split()[0] == '²':
                    floor = 2
                else:
                    floor = int(meta[1].split()[0])
            else:
                floor = 0
            if len(meta) > 3:
                if meta[2].split()[0].isnumeric():
                    area = float(meta[2].split()[0])
                    if len(meta) == 4:
                        address = meta[3].strip()
                    else:
                        address=""
                else:
                    area = 50
                    if len(meta) == 3:
                        address = meta[2].strip()
                    else:
                        address=""
            else:
                area = 50
                address=""
            date = item.find("div",class_="announcement-block__date").text.strip().split(",")[0].strip()
            price = "".join(item.find("div",class_="announcement-block__price").text.strip().split()[0:-1])
            if 'c' in price:
                price = price[0:price.find('c')]
            row = [number_of_rooms,floor,area,address,date,price]
            write_obj.writerow(row)
        
    return
print("Page parser was run successfully!")


Page parser was run successfully!


<p>In this part we create or rewrite (if already created) our csv file and pass each page to the above fuction to add new entries to our csv file.</p>

In [6]:
header = ['nuber_of_rooms','floor','area','address','date','price']
with open("estate_data.csv",'a',newline='') as obj:
    write_obj = writer(obj)
    write_obj.writerow(header)
    for i in range(1,pages+1):
        page_parser(URL+str(i))
    obj.close()

print(f"""
    All {pages} pages of https://somon.tj/nedvizhimost/prodazha-kvartir/dushanbe were parsed and the result
    is added to "estate_data.csv" file  
""")

['2-комн. квартира', ' 4 этаж', ' 44 м²', ' Сино']
['1-комн. квартира', ' 9 этаж', ' 44 м²', ' Зарафшон']
['1-комн. квартира', ' 7 этаж', ' 61 м²']
['3-комн. квартира', ' 8 этаж', ' 110 м²', ' Фирдавси']
['4-комн. квартира', ' 4 этаж', ' 71 м²', ' 9км']
['2-комн. квартира', ' 11 этаж', ' 101 м²', ' Дом печать']
['1-комн. квартира', ' 13 этаж', ' 44 м²', ' 102 мкр']
['3-комн. квартира', ' 11 этаж', ' 130 м²', ' проспект Айни', ' рядом со скорой помощь']
['2-комн. квартира', ' 3 этаж', ' 90 м²', ' проспект Айни']
['2-комн. квартира', ' 3 этаж', ' 90 м²', ' проспект Айни']
['2-комн. квартира', ' 14 этаж', ' 62 м²', ' Яккачинор']
['3-комн. квартира', ' 9 этаж', ' 90 м²', ' зарафшон']
['1-комн. квартира', ' 4 этаж', ' 45 м²', ' Cино']
['3-комн. квартира', ' 2 этаж', ' 92 м²', ' Зарафшон 2']
['2-комн. квартира', ' 3 этаж', ' 58 м²', ' проспект Айни']
['3-комн. квартира', ' 6 этаж', ' 140 м²', ' бухоро палас']
['3-комн. квартира', ' 13 этаж', ' 76 м²', ' Хувайдулоев 274/1', ' Профсоюз']
['2-к

AttributeError: 'NoneType' object has no attribute 'text'